# Quantum chemistry with Qiskit Nature: finding the interatomic distance of a hydrogen molecule

Introduction

[refs: Sharkey and qiskit-nature documentation]
[look at qiskit nature documentation for explanations]

## MoleculeInfo
We start by describing the molecule that we want to look at, in our case a hydrogen molecule:

In [4]:
distance = 0.735
molecule = f"H 0.0 0.0 0.0; H 0.0 0.0 {distance}"


* The symbols are for the two hydrogen atoms.
* We are going to vary the coordinates (in Angstrom) of the second hydrogen atom. We'll use 3 Angstrom as the initial point.
* The total charge of the hydrogen atom is 0.
* The multiplicity is 2 * S + 1, where S is the spin of the molecule due to unpaired electrons. The hydrogen molecule has no unpaired electrons, so the multiplicity is 1.

Next, we convert the molecule into electronic structure problem through the PySCFDriver:

In [2]:
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(molecule)
es_problem = driver.run()

PySCFDriver is a driver from the PySCF library for second quantization. Second quantization means converting the description of the molecule from using electronic wave functions to using Fock space. Electronic wave functions give the probability density to find an electron at a certain set of coordinates. Wave functions can be built up from eigenfunctions, functions with an allowed energy. Using these eigenfunctions, the electronic configuration can the also be described in Fock space, where we no longer look at the position we may find the electron, but at which one of the eigenstates the electron is in. [check and improve, eigenstates/functions]

In [6]:
fermionic_op = es_problem.hamiltonian.second_q_op()
n_particles = es_problem.num_particles
n_spatial_orbitals = es_problem.num_spatial_orbitals
nuclear_repulsion_energy = es_problem.nuclear_repulsion_energy

## QubitConverter
To do quantum chemistry calculations using quantum computing, operators on electrons need to be converted to operators on qubits. This is done with a QubitConverter object:

In [7]:
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper(num_particles=(1, 1))
qubit_op = mapper.map(fermionic_op)

* The specific mapper chosen uses the Jordan-Wigner transformation [why? how does it work?]. Other possible mappers are ParityMapper and BravyiKitaevMapper. [why not these? how do they work?]

## VQEUCCFactory
We are going to calculate the interatomic distance of hydrogen using the Variational Quantum Eigensolver (VQE) algorithm. VQE works in a loop, where each iteration has the following steps:
 * A quantum circuit (a sequence of qubit operations) is described by a set of parameters
 * The qubit values are measured after executing the quantum circuit.
 * Based on the qubit values, new parameters for the quantum circuit are chosen.

In our case the qubit values represent the energy of the hydrogen molecule. The goal of the loop is to find the parameters of the quantum circuit that give the lowest energy (ground state energy). The parameters of the quantum circuit model the state of the electrons of the hydrogen molecule. [how does it determine this?]


In [8]:
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

initial_state = HartreeFock(n_spatial_orbitals, n_particles, mapper)
ansatz = UCCSD(n_spatial_orbitals, n_particles, mapper, initial_state=initial_state)

The VQEUCCFactory makes a VQE instance.
* an Estimator instance is used to estimate the eigenvalues [how exactly? why?]
* the UCCSD constructs a multi-electron wave function as the ansatz, with the sum of single (S) and double (D) excitations [how exactly? what is UCC?]
* optimization in the loop is done by the Sequential Least SQuares Programming optimizer






VQEUCCFactory and Estimator, UCCSD, SLSQP
PySCFDriver
ActiveSpaceTransformer
GroundStateEigenSolver

In [17]:
from qiskit.primitives import StatevectorEstimator, Estimator
from qiskit.providers.fake_provider import GenericBackendV2

# backend = GenericBackendV2(num_qubits=5)
estimator = Estimator() #BackendEstimatorV2(backend=backend)

/tmp/ipykernel_9/2561769255.py:5: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator() #BackendEstimatorV2(backend=backend)


In [18]:

from qiskit_algorithms.optimizers import COBYLA

optimizer = COBYLA(maxiter=1)

In [19]:
from qiskit_algorithms import VQE

algo = VQE(estimator, ansatz, optimizer)
result = algo.compute_minimum_eigenvalue(qubit_op)

In [20]:
result

In [21]:
vars(result)

{'_eigenvalue': np.float64(-1.6288596769567425),
 '_aux_operators_evaluated': None,
 '_optimizer_evals': None,
 '_optimizer_time': 1.685189962387085,
 '_optimal_value': np.float64(-1.6288596769567425),
 '_optimal_point': array([ 6.04516638,  6.2124388 , -0.47337537]),
 '_optimal_parameters': {ParameterVectorElement(t[0]): np.float64(6.045166380569942),
  ParameterVectorElement(t[1]): np.float64(6.212438795189334),
  ParameterVectorElement(t[2]): np.float64(-0.4733753708743702)},
 '_optimizer_result': <qiskit_algorithms.optimizers.optimizer.OptimizerResult at 0x7f2e7dfe97f0>,
 '_optimal_circuit': <qiskit_nature.second_q.circuit.library.ansatzes.uccsd.UCCSD at 0x7f2e9815ecc0>,
 '_cost_function_evals': 1}